# About this Notebook

Bayesian probabilistic matrix factorization (BPMF) is a classical model in the recommender system field. In the following, we will discuss:

- What the BPMF is?

- How to implement BPMF mainly using Python `Numpy` with high efficiency?

- How to make data imputations with real-world spatiotemporal datasets?

If you want to know more about BPMF, please read this article:

> Ruslan Salakhutdinov, Andriy Mnih, 2008. [**Bayesian probabilistic matrix factorization using Markov chain Monte Carlo**](https://www.cs.toronto.edu/~amnih/papers/bpmf.pdf). Proceedings of the 25th International Conference on Machine Learning (*ICML 2008*), Helsinki, Finland. [[Matlab code (official)](https://www.cs.toronto.edu/~rsalakhu/BPMF.html)]

## Quick Run

This notebook is publicly available.


In [1]:
import numpy as np
from numpy.linalg import inv as inv
from numpy.random import normal as normrnd
from scipy.linalg import khatri_rao as kr_prod
from scipy.stats import wishart
from numpy.linalg import solve as solve
from scipy.linalg import cholesky as cholesky_upper
from scipy.linalg import solve_triangular as solve_ut
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [3]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [4]:
def mvnrnd_pre(mu, Lambda):
    src = normrnd(size = (mu.shape[0],))
    return solve_ut(cholesky_upper(Lambda, overwrite_a = True, check_finite = False), 
                    src, lower = False, check_finite = False, overwrite_b = True) + mu

def cov_mat(mat, mat_bar):
    mat = mat - mat_bar
    return mat.T @ mat

# Part 2: Bayesian Probabilistic Matrix Factorization (BPMF)



In [5]:
def sample_factor_w(tau_sparse_mat, tau_ind, W, X, tau, beta0 = 1, vargin = 0):
    """Sampling N-by-R factor matrix W and its hyperparameters (mu_w, Lambda_w)."""
    
    dim1, rank = W.shape
    W_bar = np.mean(W, axis = 0)
    temp = dim1 / (dim1 + beta0)
    var_mu_hyper = temp * W_bar
    var_W_hyper = inv(np.eye(rank) + cov_mat(W, W_bar) + temp * beta0 * np.outer(W_bar, W_bar))
    var_Lambda_hyper = wishart.rvs(df = dim1 + rank, scale = var_W_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim1 + beta0) * var_Lambda_hyper)
    
    if dim1 * rank ** 2 > 1e+8:
        vargin = 1
    
    if vargin == 0:
        var1 = X.T
        var2 = kr_prod(var1, var1)
        var3 = (var2 @ tau_ind.T).reshape([rank, rank, dim1]) + var_Lambda_hyper[:, :, np.newaxis]
        var4 = var1 @ tau_sparse_mat.T + (var_Lambda_hyper @ var_mu_hyper)[:, np.newaxis]
        for i in range(dim1):
            W[i, :] = mvnrnd_pre(solve(var3[:, :, i], var4[:, i]), var3[:, :, i])
    elif vargin == 1:
        for i in range(dim1):
            pos0 = np.where(sparse_mat[i, :] != 0)
            Xt = X[pos0[0], :]
            var_mu = tau[i] * Xt.T @ sparse_mat[i, pos0[0]] + var_Lambda_hyper @ var_mu_hyper
            var_Lambda = tau[i] * Xt.T @ Xt + var_Lambda_hyper
            W[i, :] = mvnrnd_pre(solve(var_Lambda, var_mu), var_Lambda)
    
    return W

In [6]:
def sample_factor_x(tau_sparse_mat, tau_ind, W, X, beta0 = 1):
    """Sampling T-by-R factor matrix X and its hyperparameters (mu_x, Lambda_x)."""
    
    dim2, rank = X.shape
    X_bar = np.mean(X, axis = 0)
    temp = dim2 / (dim2 + beta0)
    var_mu_hyper = temp * X_bar
    var_X_hyper = inv(np.eye(rank) + cov_mat(X, X_bar) + temp * beta0 * np.outer(X_bar, X_bar))
    var_Lambda_hyper = wishart.rvs(df = dim2 + rank, scale = var_X_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim2 + beta0) * var_Lambda_hyper)
    
    var1 = W.T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ tau_ind).reshape([rank, rank, dim2]) + var_Lambda_hyper[:, :, np.newaxis]
    var4 = var1 @ tau_sparse_mat + (var_Lambda_hyper @ var_mu_hyper)[:, np.newaxis]
    for t in range(dim2):
        X[t, :] = mvnrnd_pre(solve(var3[:, :, t], var4[:, t]), var3[:, :, t])

    return X

In [7]:
def sample_precision_tau(sparse_mat, mat_hat, ind):
    var_alpha = 1e-6 + 0.5 * np.sum(ind)
    var_beta = 1e-6 + 0.5 * np.sum(((sparse_mat - mat_hat) ** 2) * ind)
    return np.random.gamma(var_alpha, 1 / var_beta)

In [8]:
def BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter):
    """Bayesian Probabilistic Matrix Factorization, BPMF."""
    
    dim1, dim2 = sparse_mat.shape
    W = init["W"]
    X = init["X"]
    pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    ind = sparse_mat != 0
    pos_obs = np.where(ind)
    dense_test = dense_mat[pos_test]
    del dense_mat
    tau = 1
    W_plus = np.zeros((dim1, rank))
    X_plus = np.zeros((dim2, rank))
    temp_hat = np.zeros(sparse_mat.shape)
    show_iter = 200
    mat_hat_plus = np.zeros(sparse_mat.shape)
    for it in range(burn_iter + gibbs_iter):
        tau_ind = tau * ind
        tau_sparse_mat = tau * sparse_mat
        W = sample_factor_w(tau_sparse_mat, tau_ind, W, X, tau)
        X = sample_factor_x(tau_sparse_mat, tau_ind, W, X)
        mat_hat = W @ X.T
        tau = sample_precision_tau(sparse_mat, mat_hat, ind)
        temp_hat += mat_hat
        if (it + 1) % show_iter == 0 and it < burn_iter:
            temp_hat = temp_hat / show_iter
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat[pos_test])))
            temp_hat = np.zeros(sparse_mat.shape)
            print()
        if it + 1 > burn_iter:
            W_plus += W
            X_plus += X
            mat_hat_plus += mat_hat
    mat_hat = mat_hat_plus / gibbs_iter
    W = W_plus / gibbs_iter
    X = X_plus / gibbs_iter
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_test, mat_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_test, mat_hat[pos_test])))
    print()
    
    return mat_hat, W, X

# Part 3: Data Organization

## 1) Matrix Structure

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{f},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We express spatio-temporal dataset as a matrix $Y\in\mathbb{R}^{m\times f}$ with $m$ rows (e.g., locations) and $f$ columns (e.g., discrete time intervals),

$$Y=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{m1} & y_{m2} & \cdots & y_{mf} \\ \end{array} \right]\in\mathbb{R}^{m\times f}.$$

## 2) Tensor Structure

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{nf},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We partition each time series into intervals of predifined length $f$. We express each partitioned time series as a matrix $Y_{i}$ with $n$ rows (e.g., days) and $f$ columns (e.g., discrete time intervals per day),

$$Y_{i}=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{n1} & y_{n2} & \cdots & y_{nf} \\ \end{array} \right]\in\mathbb{R}^{n\times f},i=1,2,...,m,$$

therefore, the resulting structure is a tensor $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$.

In [9]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

dense_mat = ten2mat(dense_tensor, 0)
sparse_mat = ten2mat(sparse_tensor, 0)
del dense_tensor, sparse_tensor

In [10]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 80
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0954698
RMSE: 4.05397

Iter: 400
MAPE: 0.0956355
RMSE: 4.0647

Iter: 600
MAPE: 0.0956612
RMSE: 4.06558

Iter: 800
MAPE: 0.0956521
RMSE: 4.06583

Iter: 1000
MAPE: 0.0956608
RMSE: 4.06566

Imputation MAPE: 0.0956841
Imputation RMSE: 4.06677

Running time: 68.89 minutes


In [11]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

dense_mat = ten2mat(dense_tensor, 0)
sparse_mat = ten2mat(sparse_tensor, 0)
del dense_tensor, sparse_tensor

In [12]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 80
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.105134
RMSE: 4.41814

Iter: 400
MAPE: 0.105594
RMSE: 4.44451

Iter: 600
MAPE: 0.105577
RMSE: 4.44412

Iter: 800
MAPE: 0.105618
RMSE: 4.44589

Iter: 1000
MAPE: 0.105643
RMSE: 4.44649

Imputation MAPE: 0.105546
Imputation RMSE: 4.442

Running time: 65.92 minutes


In [13]:
pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
bpmf_ghat = dense_mat[pos_test] - mat_hat[pos_test]
np.save('bpmf_ghat.npy', bpmf_ghat)

In [14]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

dense_mat = ten2mat(dense_tensor, 0)
sparse_mat = ten2mat(sparse_tensor, 0)
del dense_tensor, sparse_tensor

In [15]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.105042
RMSE: 4.38534

Iter: 400
MAPE: 0.105037
RMSE: 4.3888

Iter: 600
MAPE: 0.105035
RMSE: 4.38874

Iter: 800
MAPE: 0.105037
RMSE: 4.38853

Iter: 1000
MAPE: 0.105034
RMSE: 4.38853

Imputation MAPE: 0.105044
Imputation RMSE: 4.38897

Running time: 7.07 minutes


In [16]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

dense_mat = ten2mat(dense_tensor, 0)
sparse_mat = ten2mat(sparse_tensor, 0)
del dense_tensor, sparse_tensor

In [17]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.111648
RMSE: 4.69297

Iter: 400
MAPE: 0.112431
RMSE: 4.75237

Iter: 600
MAPE: 0.11258
RMSE: 4.76353

Iter: 800
MAPE: 0.112554
RMSE: 4.763

Iter: 1000
MAPE: 0.112572
RMSE: 4.76332

Imputation MAPE: 0.11259
Imputation RMSE: 4.76437

Running time: 7.11 minutes


In [18]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

In [19]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 20
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0910515
RMSE: 2.20292

Iter: 400
MAPE: 0.0910523
RMSE: 2.20373

Iter: 600
MAPE: 0.0910529
RMSE: 2.20373

Iter: 800
MAPE: 0.0910515
RMSE: 2.20373

Iter: 1000
MAPE: 0.0910526
RMSE: 2.20374

Imputation MAPE: 0.0910516
Imputation RMSE: 2.2037

Running time: 61.04 minutes


In [20]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

In [21]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 20
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0939391
RMSE: 2.2701

Iter: 400
MAPE: 0.0940061
RMSE: 2.27224

Iter: 600
MAPE: 0.0940089
RMSE: 2.27224

Iter: 800
MAPE: 0.09401
RMSE: 2.2723

Iter: 1000
MAPE: 0.0940088
RMSE: 2.27225

Imputation MAPE: 0.0940071
Imputation RMSE: 2.27223

Running time: 57.92 minutes


In [22]:
pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
bpmf_lhat = dense_mat[pos_test] - mat_hat[pos_test]
np.save('bpmf_lhat.npy', bpmf_lhat)

In [23]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Non-random missing (NM)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, 30) + 0.5 - missing_rate)[:, None, :]
sparse_mat = ten2mat(sparse_tensor, 0)
del dense_tensor, sparse_tensor

In [24]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 20
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0937008
RMSE: 2.27209

Iter: 400
MAPE: 0.0938813
RMSE: 2.27677

Iter: 600
MAPE: 0.0938813
RMSE: 2.27677

Iter: 800
MAPE: 0.0938772
RMSE: 2.27679

Iter: 1000
MAPE: 0.0938803
RMSE: 2.27679

Imputation MAPE: 0.0938818
Imputation RMSE: 2.27675

Running time: 56.57 minutes


In [25]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Non-random missing (NM)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, 30) + 0.5 - missing_rate)[:, None, :]
sparse_mat = ten2mat(sparse_tensor, 0)
del dense_tensor, sparse_tensor

In [26]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 20
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0990732
RMSE: 2.40514

Iter: 400
MAPE: 0.0997365
RMSE: 2.42209

Iter: 600
MAPE: 0.0997594
RMSE: 2.42255

Iter: 800
MAPE: 0.0997472
RMSE: 2.42234

Iter: 1000
MAPE: 0.0997561
RMSE: 2.42253

Imputation MAPE: 0.0997502
Imputation RMSE: 2.42233

Running time: 56.27 minutes


# PeMS-4W

In [8]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 4 * 7), 0)
del data

missing_rate = 0.3

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [9]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 100
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.0914164
RMSE: 6.58829

Iter: 2
MAPE: 0.0599643
RMSE: 4.88216

Iter: 3
MAPE: 0.0548585
RMSE: 4.62321

Iter: 4
MAPE: 0.0522293
RMSE: 4.45302

Iter: 5
MAPE: 0.0506074
RMSE: 4.34333

Iter: 6
MAPE: 0.0505103
RMSE: 4.33636

Iter: 7
MAPE: 0.0497293
RMSE: 4.27294

Iter: 8
MAPE: 0.0497112
RMSE: 4.27192

Iter: 9
MAPE: 0.0497121
RMSE: 4.27183

Iter: 10
MAPE: 0.0497092
RMSE: 4.27135

Iter: 11
MAPE: 0.0494953
RMSE: 4.25737

Iter: 12
MAPE: 0.0487945
RMSE: 4.21461

Iter: 13
MAPE: 0.0487875
RMSE: 4.21431

Iter: 14
MAPE: 0.0487917
RMSE: 4.21424

Iter: 15
MAPE: 0.048783
RMSE: 4.21389

Iter: 16
MAPE: 0.0487636
RMSE: 4.21207

Iter: 17
MAPE: 0.0482432
RMSE: 4.17036

Iter: 18
MAPE: 0.0477558
RMSE: 4.13545

Iter: 19
MAPE: 0.0476556
RMSE: 4.12711

Iter: 20
MAPE: 0.0474919
RMSE: 4.1165

Iter: 21
MAPE: 0.047009
RMSE: 4.08491

Iter: 22
MAPE: 0.0469505
RMSE: 4.08141

Iter: 23
MAPE: 0.0464782
RMSE: 4.05224

Iter: 24
MAPE: 0.046157
RMSE: 4.02948

Iter: 25
MAPE: 0.0459739
RMSE: 4.01623

Iter: 26
MAPE

RMSE: 3.11019

Iter: 205
MAPE: 0.0353275
RMSE: 3.11011

Iter: 206
MAPE: 0.0353243
RMSE: 3.10993

Iter: 207
MAPE: 0.0353257
RMSE: 3.11005

Iter: 208
MAPE: 0.0353293
RMSE: 3.11008

Iter: 209
MAPE: 0.035322
RMSE: 3.11018

Iter: 210
MAPE: 0.0353183
RMSE: 3.10988

Iter: 211
MAPE: 0.0353267
RMSE: 3.11004

Iter: 212
MAPE: 0.0353217
RMSE: 3.11003

Iter: 213
MAPE: 0.0353203
RMSE: 3.10974

Iter: 214
MAPE: 0.0353212
RMSE: 3.1098

Iter: 215
MAPE: 0.0353095
RMSE: 3.10954

Iter: 216
MAPE: 0.0353201
RMSE: 3.10987

Iter: 217
MAPE: 0.0353234
RMSE: 3.10971

Iter: 218
MAPE: 0.0353184
RMSE: 3.10978

Iter: 219
MAPE: 0.035312
RMSE: 3.10971

Iter: 220
MAPE: 0.0353169
RMSE: 3.10972

Iter: 221
MAPE: 0.0353259
RMSE: 3.11005

Iter: 222
MAPE: 0.0353266
RMSE: 3.11003

Iter: 223
MAPE: 0.035322
RMSE: 3.10994

Iter: 224
MAPE: 0.0353162
RMSE: 3.10993

Iter: 225
MAPE: 0.0353158
RMSE: 3.1097

Iter: 226
MAPE: 0.0353239
RMSE: 3.1101

Iter: 227
MAPE: 0.0353196
RMSE: 3.10968

Iter: 228
MAPE: 0.0353278
RMSE: 3.10965

Iter: 2

RMSE: 3.09913

Iter: 406
MAPE: 0.0352005
RMSE: 3.09919

Iter: 407
MAPE: 0.0352019
RMSE: 3.09929

Iter: 408
MAPE: 0.0352047
RMSE: 3.09912

Iter: 409
MAPE: 0.0352053
RMSE: 3.09932

Iter: 410
MAPE: 0.0352096
RMSE: 3.09935

Iter: 411
MAPE: 0.0352097
RMSE: 3.09917

Iter: 412
MAPE: 0.0352021
RMSE: 3.09921

Iter: 413
MAPE: 0.0352097
RMSE: 3.09906

Iter: 414
MAPE: 0.0352084
RMSE: 3.09909

Iter: 415
MAPE: 0.0352127
RMSE: 3.09922

Iter: 416
MAPE: 0.0352095
RMSE: 3.09945

Iter: 417
MAPE: 0.0352132
RMSE: 3.09946

Iter: 418
MAPE: 0.0352131
RMSE: 3.09922

Iter: 419
MAPE: 0.0352051
RMSE: 3.09924

Iter: 420
MAPE: 0.0352075
RMSE: 3.09904

Iter: 421
MAPE: 0.0352086
RMSE: 3.09914

Iter: 422
MAPE: 0.0352103
RMSE: 3.09943

Iter: 423
MAPE: 0.0352058
RMSE: 3.09931

Iter: 424
MAPE: 0.0352069
RMSE: 3.09898

Iter: 425
MAPE: 0.0352071
RMSE: 3.09913

Iter: 426
MAPE: 0.0352116
RMSE: 3.09937

Iter: 427
MAPE: 0.035201
RMSE: 3.09893

Iter: 428
MAPE: 0.035209
RMSE: 3.09928

Iter: 429
MAPE: 0.0352096
RMSE: 3.09906

Ite

In [10]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 4 * 7), 0)
del data

missing_rate = 0.7

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [11]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 100
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.0930419
RMSE: 6.67999

Iter: 2
MAPE: 0.0880214
RMSE: 6.3715

Iter: 3
MAPE: 0.0674609
RMSE: 5.27922

Iter: 4
MAPE: 0.0606118
RMSE: 4.9269

Iter: 5
MAPE: 0.0605991
RMSE: 4.92554

Iter: 6
MAPE: 0.0606143
RMSE: 4.92566

Iter: 7
MAPE: 0.0606074
RMSE: 4.92559

Iter: 8
MAPE: 0.0606049
RMSE: 4.9256

Iter: 9
MAPE: 0.0606058
RMSE: 4.9255

Iter: 10
MAPE: 0.0606129
RMSE: 4.92559

Iter: 11
MAPE: 0.0606067
RMSE: 4.92561

Iter: 12
MAPE: 0.060601
RMSE: 4.92557

Iter: 13
MAPE: 0.0606075
RMSE: 4.92557

Iter: 14
MAPE: 0.0606072
RMSE: 4.92548

Iter: 15
MAPE: 0.0606057
RMSE: 4.92547

Iter: 16
MAPE: 0.0606023
RMSE: 4.92529

Iter: 17
MAPE: 0.0605936
RMSE: 4.92477

Iter: 18
MAPE: 0.060494
RMSE: 4.91874

Iter: 19
MAPE: 0.0575645
RMSE: 4.7655

Iter: 20
MAPE: 0.0568839
RMSE: 4.74158

Iter: 21
MAPE: 0.0568734
RMSE: 4.74119

Iter: 22
MAPE: 0.0568767
RMSE: 4.74121

Iter: 23
MAPE: 0.0568741
RMSE: 4.74069

Iter: 24
MAPE: 0.056867
RMSE: 4.74016

Iter: 25
MAPE: 0.0567485
RMSE: 4.73259

Iter: 26
MAPE: 0.

RMSE: 3.49794

Iter: 205
MAPE: 0.0402062
RMSE: 3.49754

Iter: 206
MAPE: 0.0402044
RMSE: 3.49745

Iter: 207
MAPE: 0.0401979
RMSE: 3.49673

Iter: 208
MAPE: 0.0401967
RMSE: 3.49642

Iter: 209
MAPE: 0.0401941
RMSE: 3.49659

Iter: 210
MAPE: 0.0402011
RMSE: 3.49643

Iter: 211
MAPE: 0.0401859
RMSE: 3.49567

Iter: 212
MAPE: 0.0401555
RMSE: 3.49236

Iter: 213
MAPE: 0.0401089
RMSE: 3.48745

Iter: 214
MAPE: 0.0400769
RMSE: 3.48529

Iter: 215
MAPE: 0.0400699
RMSE: 3.48511

Iter: 216
MAPE: 0.0400763
RMSE: 3.48544

Iter: 217
MAPE: 0.0400633
RMSE: 3.48508

Iter: 218
MAPE: 0.0400694
RMSE: 3.48518

Iter: 219
MAPE: 0.0400665
RMSE: 3.48522

Iter: 220
MAPE: 0.0400568
RMSE: 3.48429

Iter: 221
MAPE: 0.0400505
RMSE: 3.48358

Iter: 222
MAPE: 0.0400241
RMSE: 3.48063

Iter: 223
MAPE: 0.0400114
RMSE: 3.47887

Iter: 224
MAPE: 0.0399943
RMSE: 3.4776

Iter: 225
MAPE: 0.0399797
RMSE: 3.47721

Iter: 226
MAPE: 0.0399876
RMSE: 3.47726

Iter: 227
MAPE: 0.0399864
RMSE: 3.47732

Iter: 228
MAPE: 0.0399899
RMSE: 3.47748

It

RMSE: 3.40269

Iter: 406
MAPE: 0.039164
RMSE: 3.40246

Iter: 407
MAPE: 0.0391499
RMSE: 3.4018

Iter: 408
MAPE: 0.0391409
RMSE: 3.40182

Iter: 409
MAPE: 0.0391448
RMSE: 3.40163

Iter: 410
MAPE: 0.0391496
RMSE: 3.40153

Iter: 411
MAPE: 0.0391529
RMSE: 3.40139

Iter: 412
MAPE: 0.0391479
RMSE: 3.401

Iter: 413
MAPE: 0.0391443
RMSE: 3.40068

Iter: 414
MAPE: 0.039114
RMSE: 3.3981

Iter: 415
MAPE: 0.0390984
RMSE: 3.39696

Iter: 416
MAPE: 0.0390955
RMSE: 3.39661

Iter: 417
MAPE: 0.0390849
RMSE: 3.39671

Iter: 418
MAPE: 0.0390944
RMSE: 3.39692

Iter: 419
MAPE: 0.0390935
RMSE: 3.39685

Iter: 420
MAPE: 0.0390994
RMSE: 3.39713

Iter: 421
MAPE: 0.0390863
RMSE: 3.39637

Iter: 422
MAPE: 0.0390887
RMSE: 3.39647

Iter: 423
MAPE: 0.0390944
RMSE: 3.3971

Iter: 424
MAPE: 0.0390864
RMSE: 3.39669

Iter: 425
MAPE: 0.0390879
RMSE: 3.39641

Iter: 426
MAPE: 0.0390962
RMSE: 3.39642

Iter: 427
MAPE: 0.0390925
RMSE: 3.39673

Iter: 428
MAPE: 0.0390957
RMSE: 3.39684

Iter: 429
MAPE: 0.0390937
RMSE: 3.3968

Iter: 430

In [12]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [13]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.147755
RMSE: 10.0511

Iter: 2
MAPE: 0.0734628
RMSE: 5.55547

Iter: 3
MAPE: 0.0572216
RMSE: 4.76062

Iter: 4
MAPE: 0.0549804
RMSE: 4.65352

Iter: 5
MAPE: 0.0536734
RMSE: 4.57128

Iter: 6
MAPE: 0.053152
RMSE: 4.53206

Iter: 7
MAPE: 0.0526004
RMSE: 4.49113

Iter: 8
MAPE: 0.0525178
RMSE: 4.48873

Iter: 9
MAPE: 0.052529
RMSE: 4.48955

Iter: 10
MAPE: 0.0525066
RMSE: 4.48843

Iter: 11
MAPE: 0.0524831
RMSE: 4.48648

Iter: 12
MAPE: 0.0524598
RMSE: 4.48639

Iter: 13
MAPE: 0.0524372
RMSE: 4.48489

Iter: 14
MAPE: 0.0523841
RMSE: 4.48275

Iter: 15
MAPE: 0.0523238
RMSE: 4.47969

Iter: 16
MAPE: 0.0522807
RMSE: 4.47811

Iter: 17
MAPE: 0.0522405
RMSE: 4.47653

Iter: 18
MAPE: 0.0521798
RMSE: 4.47367

Iter: 19
MAPE: 0.0521462
RMSE: 4.47244

Iter: 20
MAPE: 0.0521034
RMSE: 4.47039

Iter: 21
MAPE: 0.0520692
RMSE: 4.46883

Iter: 22
MAPE: 0.0520352
RMSE: 4.46761

Iter: 23
MAPE: 0.0520313
RMSE: 4.4678

Iter: 24
MAPE: 0.0520191
RMSE: 4.46736

Iter: 25
MAPE: 0.0519975
RMSE: 4.46627

Iter: 26
MAPE

RMSE: 4.46528

Iter: 205
MAPE: 0.0519768
RMSE: 4.46665

Iter: 206
MAPE: 0.0519682
RMSE: 4.46572

Iter: 207
MAPE: 0.0519592
RMSE: 4.46538

Iter: 208
MAPE: 0.0519612
RMSE: 4.46587

Iter: 209
MAPE: 0.0519624
RMSE: 4.46522

Iter: 210
MAPE: 0.0519607
RMSE: 4.4656

Iter: 211
MAPE: 0.0519691
RMSE: 4.46604

Iter: 212
MAPE: 0.0519681
RMSE: 4.46561

Iter: 213
MAPE: 0.0519682
RMSE: 4.46614

Iter: 214
MAPE: 0.0519637
RMSE: 4.46547

Iter: 215
MAPE: 0.0519574
RMSE: 4.46558

Iter: 216
MAPE: 0.0519658
RMSE: 4.46631

Iter: 217
MAPE: 0.0519655
RMSE: 4.46585

Iter: 218
MAPE: 0.0519711
RMSE: 4.46628

Iter: 219
MAPE: 0.0519693
RMSE: 4.4661

Iter: 220
MAPE: 0.0519697
RMSE: 4.46589

Iter: 221
MAPE: 0.0519547
RMSE: 4.46516

Iter: 222
MAPE: 0.0519637
RMSE: 4.46546

Iter: 223
MAPE: 0.05197
RMSE: 4.46574

Iter: 224
MAPE: 0.0519639
RMSE: 4.46536

Iter: 225
MAPE: 0.0519733
RMSE: 4.4657

Iter: 226
MAPE: 0.0519775
RMSE: 4.46617

Iter: 227
MAPE: 0.0519631
RMSE: 4.46543

Iter: 228
MAPE: 0.0519643
RMSE: 4.46619

Iter: 

Iter: 406
MAPE: 0.0519671
RMSE: 4.46583

Iter: 407
MAPE: 0.0519648
RMSE: 4.46576

Iter: 408
MAPE: 0.0519586
RMSE: 4.46549

Iter: 409
MAPE: 0.0519576
RMSE: 4.46563

Iter: 410
MAPE: 0.0519723
RMSE: 4.46587

Iter: 411
MAPE: 0.0519693
RMSE: 4.46623

Iter: 412
MAPE: 0.0519668
RMSE: 4.46594

Iter: 413
MAPE: 0.0519657
RMSE: 4.46582

Iter: 414
MAPE: 0.0519657
RMSE: 4.46604

Iter: 415
MAPE: 0.051959
RMSE: 4.46531

Iter: 416
MAPE: 0.0519749
RMSE: 4.46628

Iter: 417
MAPE: 0.0519649
RMSE: 4.4658

Iter: 418
MAPE: 0.0519669
RMSE: 4.46608

Iter: 419
MAPE: 0.0519686
RMSE: 4.46589

Iter: 420
MAPE: 0.0519693
RMSE: 4.46584

Iter: 421
MAPE: 0.0519637
RMSE: 4.46538

Iter: 422
MAPE: 0.0519743
RMSE: 4.46583

Iter: 423
MAPE: 0.0519607
RMSE: 4.46554

Iter: 424
MAPE: 0.0519589
RMSE: 4.46522

Iter: 425
MAPE: 0.0519568
RMSE: 4.46513

Iter: 426
MAPE: 0.0519631
RMSE: 4.46522

Iter: 427
MAPE: 0.0519654
RMSE: 4.46575

Iter: 428
MAPE: 0.0519655
RMSE: 4.46557

Iter: 429
MAPE: 0.0519685
RMSE: 4.46575

Iter: 430
MAPE: 0.

In [14]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [15]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.265796
RMSE: 19.1895

Iter: 2
MAPE: 0.0931291
RMSE: 6.72056

Iter: 3
MAPE: 0.074345
RMSE: 5.73687

Iter: 4
MAPE: 0.0623379
RMSE: 5.07733

Iter: 5
MAPE: 0.0603819
RMSE: 5.05779

Iter: 6
MAPE: 0.0590738
RMSE: 5.04774

Iter: 7
MAPE: 0.0589727
RMSE: 5.08512

Iter: 8
MAPE: 0.0583454
RMSE: 5.05762

Iter: 9
MAPE: 0.0583448
RMSE: 5.07966

Iter: 10
MAPE: 0.0583212
RMSE: 5.08396

Iter: 11
MAPE: 0.0582654
RMSE: 5.08069

Iter: 12
MAPE: 0.0578975
RMSE: 5.0599

Iter: 13
MAPE: 0.0578613
RMSE: 5.07036

Iter: 14
MAPE: 0.0579582
RMSE: 5.08663

Iter: 15
MAPE: 0.0580146
RMSE: 5.09696

Iter: 16
MAPE: 0.0580148
RMSE: 5.10121

Iter: 17
MAPE: 0.0580712
RMSE: 5.11037

Iter: 18
MAPE: 0.0580149
RMSE: 5.10513

Iter: 19
MAPE: 0.0580052
RMSE: 5.10662

Iter: 20
MAPE: 0.0580453
RMSE: 5.11351

Iter: 21
MAPE: 0.0580822
RMSE: 5.11621

Iter: 22
MAPE: 0.0580647
RMSE: 5.11661

Iter: 23
MAPE: 0.0581034
RMSE: 5.11879

Iter: 24
MAPE: 0.0580637
RMSE: 5.12015

Iter: 25
MAPE: 0.0580618
RMSE: 5.11783

Iter: 26
MAP

RMSE: 5.11369

Iter: 205
MAPE: 0.0581593
RMSE: 5.11596

Iter: 206
MAPE: 0.0581224
RMSE: 5.11021

Iter: 207
MAPE: 0.0581467
RMSE: 5.11468

Iter: 208
MAPE: 0.0581359
RMSE: 5.11254

Iter: 209
MAPE: 0.058122
RMSE: 5.11412

Iter: 210
MAPE: 0.0581538
RMSE: 5.11192

Iter: 211
MAPE: 0.0581657
RMSE: 5.11174

Iter: 212
MAPE: 0.0581209
RMSE: 5.10907

Iter: 213
MAPE: 0.0581118
RMSE: 5.10784

Iter: 214
MAPE: 0.0581137
RMSE: 5.10772

Iter: 215
MAPE: 0.0581213
RMSE: 5.11112

Iter: 216
MAPE: 0.0581518
RMSE: 5.11532

Iter: 217
MAPE: 0.0581869
RMSE: 5.12085

Iter: 218
MAPE: 0.0581714
RMSE: 5.11776

Iter: 219
MAPE: 0.0581188
RMSE: 5.10892

Iter: 220
MAPE: 0.0581247
RMSE: 5.10816

Iter: 221
MAPE: 0.0581598
RMSE: 5.11428

Iter: 222
MAPE: 0.0581286
RMSE: 5.11267

Iter: 223
MAPE: 0.0581588
RMSE: 5.11577

Iter: 224
MAPE: 0.0581298
RMSE: 5.11325

Iter: 225
MAPE: 0.0581734
RMSE: 5.12007

Iter: 226
MAPE: 0.058165
RMSE: 5.11643

Iter: 227
MAPE: 0.0581671
RMSE: 5.11541

Iter: 228
MAPE: 0.058157
RMSE: 5.11415

Iter

MAPE: 0.058141
RMSE: 5.11208

Iter: 406
MAPE: 0.058124
RMSE: 5.11321

Iter: 407
MAPE: 0.0581518
RMSE: 5.113

Iter: 408
MAPE: 0.0581393
RMSE: 5.11457

Iter: 409
MAPE: 0.0581486
RMSE: 5.11438

Iter: 410
MAPE: 0.0581365
RMSE: 5.1111

Iter: 411
MAPE: 0.0581041
RMSE: 5.10738

Iter: 412
MAPE: 0.0581599
RMSE: 5.11777

Iter: 413
MAPE: 0.058158
RMSE: 5.11706

Iter: 414
MAPE: 0.0581703
RMSE: 5.11715

Iter: 415
MAPE: 0.0581649
RMSE: 5.11513

Iter: 416
MAPE: 0.0581544
RMSE: 5.11511

Iter: 417
MAPE: 0.0581496
RMSE: 5.11503

Iter: 418
MAPE: 0.0581205
RMSE: 5.10778

Iter: 419
MAPE: 0.0581317
RMSE: 5.1106

Iter: 420
MAPE: 0.0581464
RMSE: 5.11054

Iter: 421
MAPE: 0.0580997
RMSE: 5.10644

Iter: 422
MAPE: 0.0581607
RMSE: 5.11224

Iter: 423
MAPE: 0.0581667
RMSE: 5.1145

Iter: 424
MAPE: 0.0581307
RMSE: 5.10723

Iter: 425
MAPE: 0.0580993
RMSE: 5.1057

Iter: 426
MAPE: 0.0581461
RMSE: 5.10805

Iter: 427
MAPE: 0.0581362
RMSE: 5.1083

Iter: 428
MAPE: 0.0581252
RMSE: 5.10908

Iter: 429
MAPE: 0.058113
RMSE: 5.107

# PeMS-8W

In [20]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 8 * 7), 0)
del data

missing_rate = 0.3

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [21]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 100
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.0968542
RMSE: 6.84772

Iter: 2
MAPE: 0.0601627
RMSE: 4.94326

Iter: 3
MAPE: 0.0536714
RMSE: 4.56051

Iter: 4
MAPE: 0.0519266
RMSE: 4.44854

Iter: 5
MAPE: 0.0516433
RMSE: 4.42952

Iter: 6
MAPE: 0.0510477
RMSE: 4.38518

Iter: 7
MAPE: 0.0507995
RMSE: 4.36934

Iter: 8
MAPE: 0.0504458
RMSE: 4.34578

Iter: 9
MAPE: 0.0504403
RMSE: 4.34563

Iter: 10
MAPE: 0.0504385
RMSE: 4.34536

Iter: 11
MAPE: 0.0503829
RMSE: 4.34214

Iter: 12
MAPE: 0.0499215
RMSE: 4.31329

Iter: 13
MAPE: 0.0499078
RMSE: 4.3127

Iter: 14
MAPE: 0.0498941
RMSE: 4.31214

Iter: 15
MAPE: 0.049773
RMSE: 4.30576

Iter: 16
MAPE: 0.0490059
RMSE: 4.259

Iter: 17
MAPE: 0.0488951
RMSE: 4.24898

Iter: 18
MAPE: 0.0485417
RMSE: 4.2239

Iter: 19
MAPE: 0.0485094
RMSE: 4.22119

Iter: 20
MAPE: 0.0485117
RMSE: 4.22111

Iter: 21
MAPE: 0.0485083
RMSE: 4.22088

Iter: 22
MAPE: 0.0484871
RMSE: 4.21964

Iter: 23
MAPE: 0.0482189
RMSE: 4.19844

Iter: 24
MAPE: 0.0478171
RMSE: 4.17356

Iter: 25
MAPE: 0.0477843
RMSE: 4.17131

Iter: 26
MAPE:

Iter: 205
MAPE: 0.0397322
RMSE: 3.47763

Iter: 206
MAPE: 0.0397222
RMSE: 3.47648

Iter: 207
MAPE: 0.0397026
RMSE: 3.47406

Iter: 208
MAPE: 0.0396992
RMSE: 3.47365

Iter: 209
MAPE: 0.0396963
RMSE: 3.47366

Iter: 210
MAPE: 0.0396969
RMSE: 3.47349

Iter: 211
MAPE: 0.0396972
RMSE: 3.47355

Iter: 212
MAPE: 0.0396934
RMSE: 3.47318

Iter: 213
MAPE: 0.0396918
RMSE: 3.47294

Iter: 214
MAPE: 0.0396872
RMSE: 3.47274

Iter: 215
MAPE: 0.0396904
RMSE: 3.47284

Iter: 216
MAPE: 0.0396878
RMSE: 3.4728

Iter: 217
MAPE: 0.0396888
RMSE: 3.47279

Iter: 218
MAPE: 0.0396897
RMSE: 3.47269

Iter: 219
MAPE: 0.0396907
RMSE: 3.47268

Iter: 220
MAPE: 0.0396891
RMSE: 3.47257

Iter: 221
MAPE: 0.0396848
RMSE: 3.47262

Iter: 222
MAPE: 0.0396865
RMSE: 3.47248

Iter: 223
MAPE: 0.0396899
RMSE: 3.47271

Iter: 224
MAPE: 0.0396883
RMSE: 3.47275

Iter: 225
MAPE: 0.0396846
RMSE: 3.47233

Iter: 226
MAPE: 0.039688
RMSE: 3.47245

Iter: 227
MAPE: 0.0396861
RMSE: 3.47247

Iter: 228
MAPE: 0.0396848
RMSE: 3.4725

Iter: 229
MAPE: 0.0

Iter: 406
MAPE: 0.0396278
RMSE: 3.4678

Iter: 407
MAPE: 0.0396304
RMSE: 3.46813

Iter: 408
MAPE: 0.0396286
RMSE: 3.46813

Iter: 409
MAPE: 0.0396282
RMSE: 3.46788

Iter: 410
MAPE: 0.039627
RMSE: 3.46789

Iter: 411
MAPE: 0.0396263
RMSE: 3.46783

Iter: 412
MAPE: 0.0396254
RMSE: 3.46793

Iter: 413
MAPE: 0.039622
RMSE: 3.46787

Iter: 414
MAPE: 0.0396256
RMSE: 3.46788

Iter: 415
MAPE: 0.0396208
RMSE: 3.46768

Iter: 416
MAPE: 0.0396286
RMSE: 3.46804

Iter: 417
MAPE: 0.0396265
RMSE: 3.46804

Iter: 418
MAPE: 0.0396252
RMSE: 3.46795

Iter: 419
MAPE: 0.0396303
RMSE: 3.46811

Iter: 420
MAPE: 0.0396219
RMSE: 3.46766

Iter: 421
MAPE: 0.0396272
RMSE: 3.46808

Iter: 422
MAPE: 0.039627
RMSE: 3.468

Iter: 423
MAPE: 0.0396315
RMSE: 3.46796

Iter: 424
MAPE: 0.0396245
RMSE: 3.46805

Iter: 425
MAPE: 0.0396265
RMSE: 3.46795

Iter: 426
MAPE: 0.0396312
RMSE: 3.46795

Iter: 427
MAPE: 0.039626
RMSE: 3.46786

Iter: 428
MAPE: 0.039624
RMSE: 3.46798

Iter: 429
MAPE: 0.0396238
RMSE: 3.46776

Iter: 430
MAPE: 0.039625

In [22]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 8 * 7), 0)
del data

missing_rate = 0.7

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [23]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 100
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.0991596
RMSE: 6.99167

Iter: 2
MAPE: 0.0784775
RMSE: 5.82079

Iter: 3
MAPE: 0.0638669
RMSE: 5.14709

Iter: 4
MAPE: 0.0638322
RMSE: 5.14513

Iter: 5
MAPE: 0.0637671
RMSE: 5.14105

Iter: 6
MAPE: 0.060673
RMSE: 4.97382

Iter: 7
MAPE: 0.0597329
RMSE: 4.93921

Iter: 8
MAPE: 0.0597067
RMSE: 4.93872

Iter: 9
MAPE: 0.0596796
RMSE: 4.9384

Iter: 10
MAPE: 0.0596869
RMSE: 4.9384

Iter: 11
MAPE: 0.0596842
RMSE: 4.93842

Iter: 12
MAPE: 0.0596773
RMSE: 4.93827

Iter: 13
MAPE: 0.0596798
RMSE: 4.93809

Iter: 14
MAPE: 0.0596569
RMSE: 4.93683

Iter: 15
MAPE: 0.0591685
RMSE: 4.90901

Iter: 16
MAPE: 0.0574684
RMSE: 4.82021

Iter: 17
MAPE: 0.0574781
RMSE: 4.81903

Iter: 18
MAPE: 0.0574959
RMSE: 4.8188

Iter: 19
MAPE: 0.0575068
RMSE: 4.81866

Iter: 20
MAPE: 0.05752
RMSE: 4.81854

Iter: 21
MAPE: 0.0575284
RMSE: 4.81854

Iter: 22
MAPE: 0.0575317
RMSE: 4.81851

Iter: 23
MAPE: 0.0575331
RMSE: 4.81849

Iter: 24
MAPE: 0.0575396
RMSE: 4.8186

Iter: 25
MAPE: 0.0575388
RMSE: 4.81843

Iter: 26
MAPE: 0

Iter: 205
MAPE: 0.0437973
RMSE: 3.80561

Iter: 206
MAPE: 0.0437862
RMSE: 3.80508

Iter: 207
MAPE: 0.0437848
RMSE: 3.80471

Iter: 208
MAPE: 0.0437758
RMSE: 3.80424

Iter: 209
MAPE: 0.0437702
RMSE: 3.80356

Iter: 210
MAPE: 0.043752
RMSE: 3.8009

Iter: 211
MAPE: 0.0437473
RMSE: 3.8005

Iter: 212
MAPE: 0.0437383
RMSE: 3.80016

Iter: 213
MAPE: 0.043733
RMSE: 3.79988

Iter: 214
MAPE: 0.0437401
RMSE: 3.79984

Iter: 215
MAPE: 0.0437321
RMSE: 3.79947

Iter: 216
MAPE: 0.0437301
RMSE: 3.79942

Iter: 217
MAPE: 0.0437307
RMSE: 3.79909

Iter: 218
MAPE: 0.0437249
RMSE: 3.79917

Iter: 219
MAPE: 0.0437158
RMSE: 3.79824

Iter: 220
MAPE: 0.0436967
RMSE: 3.79554

Iter: 221
MAPE: 0.0436952
RMSE: 3.7949

Iter: 222
MAPE: 0.0436745
RMSE: 3.79381

Iter: 223
MAPE: 0.0436429
RMSE: 3.79082

Iter: 224
MAPE: 0.0436408
RMSE: 3.79015

Iter: 225
MAPE: 0.0436362
RMSE: 3.78984

Iter: 226
MAPE: 0.0436273
RMSE: 3.7893

Iter: 227
MAPE: 0.0436007
RMSE: 3.78643

Iter: 228
MAPE: 0.0435916
RMSE: 3.78531

Iter: 229
MAPE: 0.0435

RMSE: 3.71816

Iter: 407
MAPE: 0.0428952
RMSE: 3.71774

Iter: 408
MAPE: 0.0428861
RMSE: 3.71615

Iter: 409
MAPE: 0.0428835
RMSE: 3.71516

Iter: 410
MAPE: 0.042881
RMSE: 3.71515

Iter: 411
MAPE: 0.0428731
RMSE: 3.71521

Iter: 412
MAPE: 0.042872
RMSE: 3.71508

Iter: 413
MAPE: 0.0428725
RMSE: 3.71492

Iter: 414
MAPE: 0.0428705
RMSE: 3.715

Iter: 415
MAPE: 0.0428717
RMSE: 3.71509

Iter: 416
MAPE: 0.0428705
RMSE: 3.71524

Iter: 417
MAPE: 0.0428729
RMSE: 3.71499

Iter: 418
MAPE: 0.0428679
RMSE: 3.715

Iter: 419
MAPE: 0.0428666
RMSE: 3.71479

Iter: 420
MAPE: 0.0428663
RMSE: 3.71467

Iter: 421
MAPE: 0.0428659
RMSE: 3.71472

Iter: 422
MAPE: 0.0428677
RMSE: 3.71492

Iter: 423
MAPE: 0.0428675
RMSE: 3.71498

Iter: 424
MAPE: 0.0428569
RMSE: 3.71432

Iter: 425
MAPE: 0.0428639
RMSE: 3.71479

Iter: 426
MAPE: 0.0428587
RMSE: 3.71399

Iter: 427
MAPE: 0.0428373
RMSE: 3.71256

Iter: 428
MAPE: 0.0428352
RMSE: 3.71181

Iter: 429
MAPE: 0.0428346
RMSE: 3.71154

Iter: 430
MAPE: 0.0428304
RMSE: 3.71176

Iter: 4

In [16]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [17]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.125603
RMSE: 8.38345

Iter: 2
MAPE: 0.06631
RMSE: 5.24349

Iter: 3
MAPE: 0.0593861
RMSE: 4.92073

Iter: 4
MAPE: 0.0558042
RMSE: 4.72064

Iter: 5
MAPE: 0.0549001
RMSE: 4.66421

Iter: 6
MAPE: 0.0541586
RMSE: 4.61508

Iter: 7
MAPE: 0.0534479
RMSE: 4.56849

Iter: 8
MAPE: 0.0534407
RMSE: 4.56921

Iter: 9
MAPE: 0.0534498
RMSE: 4.56964

Iter: 10
MAPE: 0.0534533
RMSE: 4.57025

Iter: 11
MAPE: 0.0534493
RMSE: 4.57011

Iter: 12
MAPE: 0.0534559
RMSE: 4.57059

Iter: 13
MAPE: 0.0534504
RMSE: 4.57045

Iter: 14
MAPE: 0.0534491
RMSE: 4.57058

Iter: 15
MAPE: 0.0534511
RMSE: 4.57092

Iter: 16
MAPE: 0.053456
RMSE: 4.57103

Iter: 17
MAPE: 0.0534497
RMSE: 4.57091

Iter: 18
MAPE: 0.0534562
RMSE: 4.57119

Iter: 19
MAPE: 0.0534501
RMSE: 4.57115

Iter: 20
MAPE: 0.0534525
RMSE: 4.57125

Iter: 21
MAPE: 0.0534554
RMSE: 4.57137

Iter: 22
MAPE: 0.0534594
RMSE: 4.57155

Iter: 23
MAPE: 0.0534548
RMSE: 4.5715

Iter: 24
MAPE: 0.0534546
RMSE: 4.57144

Iter: 25
MAPE: 0.0534554
RMSE: 4.57136

Iter: 26
MAPE:

RMSE: 4.5684

Iter: 205
MAPE: 0.0534378
RMSE: 4.56842

Iter: 206
MAPE: 0.0534374
RMSE: 4.56821

Iter: 207
MAPE: 0.0534422
RMSE: 4.56854

Iter: 208
MAPE: 0.0534366
RMSE: 4.56809

Iter: 209
MAPE: 0.0534373
RMSE: 4.56825

Iter: 210
MAPE: 0.0534385
RMSE: 4.56815

Iter: 211
MAPE: 0.0534342
RMSE: 4.56821

Iter: 212
MAPE: 0.0534396
RMSE: 4.56813

Iter: 213
MAPE: 0.0534352
RMSE: 4.56828

Iter: 214
MAPE: 0.0534403
RMSE: 4.56869

Iter: 215
MAPE: 0.0534391
RMSE: 4.56846

Iter: 216
MAPE: 0.0534379
RMSE: 4.56844

Iter: 217
MAPE: 0.0534356
RMSE: 4.56827

Iter: 218
MAPE: 0.0534381
RMSE: 4.56824

Iter: 219
MAPE: 0.0534399
RMSE: 4.56851

Iter: 220
MAPE: 0.053438
RMSE: 4.56824

Iter: 221
MAPE: 0.0534382
RMSE: 4.56836

Iter: 222
MAPE: 0.0534428
RMSE: 4.56833

Iter: 223
MAPE: 0.0534401
RMSE: 4.56864

Iter: 224
MAPE: 0.0534344
RMSE: 4.56841

Iter: 225
MAPE: 0.0534346
RMSE: 4.56835

Iter: 226
MAPE: 0.0534367
RMSE: 4.56824

Iter: 227
MAPE: 0.0534355
RMSE: 4.56824

Iter: 228
MAPE: 0.0534356
RMSE: 4.56833

Ite

RMSE: 4.56822

Iter: 406
MAPE: 0.0534423
RMSE: 4.56839

Iter: 407
MAPE: 0.053437
RMSE: 4.56818

Iter: 408
MAPE: 0.0534387
RMSE: 4.56855

Iter: 409
MAPE: 0.0534368
RMSE: 4.56816

Iter: 410
MAPE: 0.0534317
RMSE: 4.56811

Iter: 411
MAPE: 0.0534384
RMSE: 4.56821

Iter: 412
MAPE: 0.0534374
RMSE: 4.56831

Iter: 413
MAPE: 0.0534357
RMSE: 4.56821

Iter: 414
MAPE: 0.0534407
RMSE: 4.56848

Iter: 415
MAPE: 0.0534389
RMSE: 4.56814

Iter: 416
MAPE: 0.053439
RMSE: 4.56843

Iter: 417
MAPE: 0.0534368
RMSE: 4.56814

Iter: 418
MAPE: 0.0534329
RMSE: 4.56809

Iter: 419
MAPE: 0.053436
RMSE: 4.56835

Iter: 420
MAPE: 0.0534393
RMSE: 4.56841

Iter: 421
MAPE: 0.0534328
RMSE: 4.56823

Iter: 422
MAPE: 0.0534394
RMSE: 4.56846

Iter: 423
MAPE: 0.0534386
RMSE: 4.56842

Iter: 424
MAPE: 0.0534344
RMSE: 4.56827

Iter: 425
MAPE: 0.0534399
RMSE: 4.56853

Iter: 426
MAPE: 0.0534335
RMSE: 4.56815

Iter: 427
MAPE: 0.0534339
RMSE: 4.5682

Iter: 428
MAPE: 0.0534295
RMSE: 4.56806

Iter: 429
MAPE: 0.0534417
RMSE: 4.5685

Iter: 

In [18]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [19]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
mat_hat, W, X = BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 1
MAPE: 0.198605
RMSE: 13.6923

Iter: 2
MAPE: 0.0990871
RMSE: 7.00511

Iter: 3
MAPE: 0.0769632
RMSE: 5.8668

Iter: 4
MAPE: 0.0644758
RMSE: 5.19703

Iter: 5
MAPE: 0.0597161
RMSE: 4.95975

Iter: 6
MAPE: 0.0576576
RMSE: 4.84735

Iter: 7
MAPE: 0.0572709
RMSE: 4.82567

Iter: 8
MAPE: 0.0564141
RMSE: 4.77022

Iter: 9
MAPE: 0.0558536
RMSE: 4.73569

Iter: 10
MAPE: 0.0558808
RMSE: 4.74256

Iter: 11
MAPE: 0.0558688
RMSE: 4.7439

Iter: 12
MAPE: 0.055854
RMSE: 4.74361

Iter: 13
MAPE: 0.0558348
RMSE: 4.74315

Iter: 14
MAPE: 0.0558148
RMSE: 4.74289

Iter: 15
MAPE: 0.0557884
RMSE: 4.74127

Iter: 16
MAPE: 0.0557742
RMSE: 4.74074

Iter: 17
MAPE: 0.0557524
RMSE: 4.74027

Iter: 18
MAPE: 0.0557425
RMSE: 4.73934

Iter: 19
MAPE: 0.0557225
RMSE: 4.73864

Iter: 20
MAPE: 0.0556972
RMSE: 4.73783

Iter: 21
MAPE: 0.0556756
RMSE: 4.73621

Iter: 22
MAPE: 0.0554857
RMSE: 4.72412

Iter: 23
MAPE: 0.0552514
RMSE: 4.71115

Iter: 24
MAPE: 0.0553371
RMSE: 4.71796

Iter: 25
MAPE: 0.055372
RMSE: 4.72109

Iter: 26
MAPE:

RMSE: 4.73021

Iter: 205
MAPE: 0.0554344
RMSE: 4.73004

Iter: 206
MAPE: 0.0554296
RMSE: 4.73018

Iter: 207
MAPE: 0.0554348
RMSE: 4.73041

Iter: 208
MAPE: 0.0554329
RMSE: 4.73047

Iter: 209
MAPE: 0.0554309
RMSE: 4.73054

Iter: 210
MAPE: 0.0554395
RMSE: 4.73097

Iter: 211
MAPE: 0.0554455
RMSE: 4.73143

Iter: 212
MAPE: 0.0554396
RMSE: 4.7305

Iter: 213
MAPE: 0.0554323
RMSE: 4.73057

Iter: 214
MAPE: 0.0554254
RMSE: 4.72983

Iter: 215
MAPE: 0.0554357
RMSE: 4.73028

Iter: 216
MAPE: 0.055437
RMSE: 4.73044

Iter: 217
MAPE: 0.0554388
RMSE: 4.7307

Iter: 218
MAPE: 0.0554393
RMSE: 4.73038

Iter: 219
MAPE: 0.0554366
RMSE: 4.72989

Iter: 220
MAPE: 0.0554485
RMSE: 4.73101

Iter: 221
MAPE: 0.0554307
RMSE: 4.72998

Iter: 222
MAPE: 0.0554401
RMSE: 4.7303

Iter: 223
MAPE: 0.055431
RMSE: 4.72998

Iter: 224
MAPE: 0.0554388
RMSE: 4.73027

Iter: 225
MAPE: 0.0554334
RMSE: 4.73034

Iter: 226
MAPE: 0.0554289
RMSE: 4.72986

Iter: 227
MAPE: 0.0554306
RMSE: 4.72957

Iter: 228
MAPE: 0.0554216
RMSE: 4.72961

Iter: 

RMSE: 4.72977

Iter: 406
MAPE: 0.0554382
RMSE: 4.73022

Iter: 407
MAPE: 0.0554256
RMSE: 4.7298

Iter: 408
MAPE: 0.0554276
RMSE: 4.73005

Iter: 409
MAPE: 0.0554369
RMSE: 4.73019

Iter: 410
MAPE: 0.0554375
RMSE: 4.73108

Iter: 411
MAPE: 0.0554354
RMSE: 4.73012

Iter: 412
MAPE: 0.0554384
RMSE: 4.73032

Iter: 413
MAPE: 0.0554375
RMSE: 4.73012

Iter: 414
MAPE: 0.0554274
RMSE: 4.72993

Iter: 415
MAPE: 0.0554324
RMSE: 4.73002

Iter: 416
MAPE: 0.0554372
RMSE: 4.73009

Iter: 417
MAPE: 0.0554413
RMSE: 4.73006

Iter: 418
MAPE: 0.0554347
RMSE: 4.72968

Iter: 419
MAPE: 0.0554288
RMSE: 4.7297

Iter: 420
MAPE: 0.0554295
RMSE: 4.72969

Iter: 421
MAPE: 0.0554352
RMSE: 4.73007

Iter: 422
MAPE: 0.0554358
RMSE: 4.7303

Iter: 423
MAPE: 0.0554405
RMSE: 4.73045

Iter: 424
MAPE: 0.055434
RMSE: 4.72993

Iter: 425
MAPE: 0.0554363
RMSE: 4.72968

Iter: 426
MAPE: 0.055435
RMSE: 4.72962

Iter: 427
MAPE: 0.0554299
RMSE: 4.72958

Iter: 428
MAPE: 0.0554244
RMSE: 4.72957

Iter: 429
MAPE: 0.055436
RMSE: 4.73019

Iter: 4

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>